# Populate Config Cache

**Purpose:** Parse clinical_data_standards.yaml and populate the md_config_cache Delta table.

**When to run:**
- After modifying clinical_data_standards.yaml
- After initial deployment
- Can be run manually or as part of deployment pipeline

**Performance benefit:**
- Setup task reads from Delta (~2-5 sec) instead of parsing YAML (~30-60 sec)
- Reduces serverless cold start impact


In [ ]:
# Cell 1: Imports
import json
import yaml
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

print("="*80)
print("POPULATE CONFIG CACHE")
print("="*80)


In [ ]:
# Cell 2: Get Parameters
dbutils.widgets.text("workspace_file_path", "", "Workspace File Path")
dbutils.widgets.text("config_version", "1.0.0", "Config Version")
dbutils.widgets.text("updated_by", "system", "Updated By")

workspace_file_path = dbutils.widgets.get("workspace_file_path")
config_version = dbutils.widgets.get("config_version")
updated_by = dbutils.widgets.get("updated_by")

# If workspace_file_path not provided, try to detect it
if not workspace_file_path:
    # Try to get from current notebook path
    try:
        notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        # Assume config is at ../../config relative to notebooks/data_engineering/common/
        workspace_file_path = "/".join(notebook_path.split("/")[:-4])
        print(f"Auto-detected workspace path: {workspace_file_path}")
    except:
        raise ValueError("workspace_file_path parameter required")

config_path = f"{workspace_file_path}/config/clinical_data_standards.yaml"
print(f"Config path: {config_path}")
print(f"Version: {config_version}")
print(f"Updated by: {updated_by}")


In [ ]:
# Cell 3: Read and Parse YAML Config
print(f"\n{'='*60}")
print("Reading YAML configuration...")
print(f"{'='*60}")

# Read the YAML file
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print(f"Loaded config with {len(config)} top-level sections:")
for key in config.keys():
    print(f"  - {key}")


In [ ]:
# Cell 4: Prepare Config Sections for Cache
print(f"\n{'='*60}")
print("Preparing config sections...")
print(f"{'='*60}")

now = datetime.now()
config_key = "clinical_data_standards"

# Define sections to cache
sections_to_cache = [
    "globals",
    "services",
    "versioning",
    "streaming",
    "pipelines",
    "validation"
]

cache_records = []
for section in sections_to_cache:
    section_data = config.get(section, {})
    json_str = json.dumps(section_data)
    
    record = {
        "config_key": config_key,
        "config_section": section,
        "config_json": json_str,
        "config_version": config_version,
        "updated_ts": now,
        "updated_by": updated_by
    }
    cache_records.append(record)
    
    # Show preview
    print(f"  {section}: {len(json_str)} bytes")

print(f"\nPrepared {len(cache_records)} sections for caching")


In [ ]:
# Cell 5: Get Target Table from Config
catalog = config.get('globals', {}).get('catalog')
bronze_schema = config.get('globals', {}).get('bronze_schema')

# Fail fast if catalog or schema not defined - never use hardcoded defaults
if not catalog:
    raise ValueError("Catalog not defined in config/clinical_data_standards.yaml globals.catalog")
if not bronze_schema:
    raise ValueError("Bronze schema not defined in config/clinical_data_standards.yaml globals.bronze_schema")

cache_table = f"{catalog}.{bronze_schema}.md_config_cache"
print(f"Target cache table: {cache_table}")


In [ ]:
# Cell 6: Create Table if Not Exists
schema = StructType([
    StructField("config_key", StringType(), False),
    StructField("config_section", StringType(), False),
    StructField("config_json", StringType(), True),
    StructField("config_version", StringType(), True),
    StructField("updated_ts", TimestampType(), True),
    StructField("updated_by", StringType(), True)
])

if not spark.catalog.tableExists(cache_table):
    print(f"Creating cache table: {cache_table}")
    empty_df = spark.createDataFrame([], schema=schema)
    empty_df.write.format("delta").mode("overwrite").saveAsTable(cache_table)
    print(f"Created {cache_table}")
else:
    print(f"Cache table exists: {cache_table}")


In [ ]:
# Cell 7: Upsert Config to Cache Table
print(f"\n{'='*60}")
print("Upserting config to cache table...")
print(f"{'='*60}")

# Create DataFrame from records
cache_df = spark.createDataFrame(cache_records, schema=schema)

# Create temp view for merge
cache_df.createOrReplaceTempView("config_updates")

# Upsert using MERGE
merge_sql = f"""
MERGE INTO {cache_table} AS target
USING config_updates AS source
ON target.config_key = source.config_key AND target.config_section = source.config_section
WHEN MATCHED THEN
    UPDATE SET
        config_json = source.config_json,
        config_version = source.config_version,
        updated_ts = source.updated_ts,
        updated_by = source.updated_by
WHEN NOT MATCHED THEN
    INSERT (config_key, config_section, config_json, config_version, updated_ts, updated_by)
    VALUES (source.config_key, source.config_section, source.config_json, source.config_version, source.updated_ts, source.updated_by)
"""

spark.sql(merge_sql)
print(f"Upserted {len(cache_records)} config sections")


In [ ]:
# Cell 8: Verify Cache
print(f"\n{'='*60}")
print("Verifying cache...")
print(f"{'='*60}")

result_df = spark.table(cache_table).filter(
    F.col("config_key") == config_key
).select(
    "config_section",
    F.length("config_json").alias("json_bytes"),
    "config_version",
    "updated_ts"
).orderBy("config_section")

result_df.show(truncate=False)

total_sections = result_df.count()
print(f"\n✅ Config cache populated successfully!")
print(f"   Sections cached: {total_sections}")
print(f"   Version: {config_version}")
print(f"\n📋 Setup notebooks can now read from: {cache_table}")


In [ ]:
# Cell 9: Test Read Performance
import time

print(f"\n{'='*60}")
print("Testing read performance...")
print(f"{'='*60}")

start = time.time()

# Simulate what nb_setup will do
config_df = spark.table(cache_table).filter(
    F.col("config_key") == "clinical_data_standards"
).select("config_section", "config_json")

configs = {row.config_section: json.loads(row.config_json) for row in config_df.collect()}

elapsed = time.time() - start

print(f"\n⏱️  Read time: {elapsed:.3f} seconds")
print(f"   Loaded {len(configs)} sections: {list(configs.keys())}")

if elapsed < 5:
    print(f"\n✅ Performance target met! (< 5 seconds)")
else:
    print(f"\n⚠️  Performance may need optimization ({elapsed:.1f}s > 5s target)")
